# Machine Learning in Chess to Study Patterns in Chess Games

## Import Libraries

In [10]:
import pandas as pd
import dask.dataframe as dd


## Load the Data

In [4]:
DATA_PATH = "/Users/isaac/Downloads/ChessDBs/lichess_db_standard_rated_2022.parquet"
# Check whether the DataFrame has been loaded correctly.
df = dd.read_parquet(DATA_PATH)
print(f"Loaded {len(df)} Rated Bullet, Rated Blitz, and Rated Rapid games from 2022.\n")
display(df)
display(df.head())


Loaded 63927025 Rated Bullet, Rated Blitz, and Rated Rapid games from 2022.



,UTCDate,UTCTime,Event,TimeControl,Result,Termination,ECO,Opening,White,Black,WhiteElo,BlackElo,Site
npartitions=153,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,int64,int64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


,UTCDate,UTCTime,Event,TimeControl,Result,Termination,ECO,Opening,White,Black,WhiteElo,BlackElo,Site
0,2022.01.01,00:00:11,Rated Bullet game,120+1,0-1,Normal,D02,Queen's Pawn Game: Chigorin Variation,Kike73,Aox066,1255,1252,https://lichess.org/BPNsj2LF
1,2022.01.01,00:00:11,Rated Bullet game,120+1,1-0,Normal,C40,King's Pawn Game: Busch-Gass Gambit,schachueli,HiddenEyes,1346,1308,https://lichess.org/ooDjyGrm
2,2022.01.01,00:00:11,Rated Bullet game,120+1,0-1,Time forfeit,C02,"French Defense: Advance Variation, Nimzowitsch...",Renf7,Euphorbe92,859,964,https://lichess.org/T4TIdZZM
3,2022.01.01,00:00:11,Rated Bullet game,120+1,0-1,Time forfeit,B10,Caro-Kann Defense: Accelerated Panov Attack,Ralza63,FN2718,1769,1702,https://lichess.org/Oz5OFohi
4,2022.01.01,00:00:11,Rated Bullet game,120+1,1-0,Time forfeit,A41,Queen's Pawn Game,arkchess1963,sirkkeli,2146,2196,https://lichess.org/iehbAAzy


## Get Most Popular Openings

In [5]:
display(df["Opening"].value_counts().compute())

Modern Defense                                                     1643325
Queen's Pawn Game                                                  1636105
Scandinavian Defense: Mieses-Kotroc Variation                      1400483
Caro-Kann Defense                                                  1231717
Philidor Defense                                                   1154531
                                                                    ...   
Queen's Gambit Accepted: Alekhine Defense, Alatortsev Variation          1
King's Gambit Accepted: Bishop's Gambit, Chigorin's Attack               1
Italian Game: Evans Gambit, Göring Attack                                1
Ruy Lopez: Open, Bernstein Variation, Luther Line                        1
Vienna Game: Stanley Variation, Eifel Gambit                             1
Name: Opening, Length: 2997, dtype: int64

## Get Most Popular Opening Categories

In [6]:
display(df["ECO"].value_counts().compute())

A00    3951040
A40    3527282
B01    3384668
D00    2822607
B00    2794027
        ...   
A79         53
D68         49
E96         24
D65         15
D99          1
Name: ECO, Length: 497, dtype: int64

## Get Elo Distribution of Players

In [14]:
white_elo_count = df["WhiteElo"].value_counts().compute()
black_elo_count = df["BlackElo"].value_counts().compute()
# Merge the counts into one, as the player's side isn't important.
elo_count = pd.concat([white_elo_count, black_elo_count], axis=1).fillna(0).astype(int)
elo_count = elo_count.sum(axis=1)
display(elo_count)


1500    446870
1743    130036
1732    129679
1735    128645
1725    129228
         ...  
3280         1
3267         1
3240         1
3290         1
3485         1
Length: 2721, dtype: int64